In [27]:
import sklearn as sk
import numpy as np
import pandas as pd

In [41]:
df = pd.read_csv('/mnt/data/baseline.csv', header=0, index_col=0)
df.churn.value_counts()

N    7607
Y     397
Name: churn, dtype: int64

In [42]:
create_dummies = pd.get_dummies(df['churn'],prefix='churn', drop_first=True)  
df=pd.concat([df,create_dummies],axis=1)
df = df.drop(['churn'], axis=1)

In [43]:
from sklearn.utils import resample

# Separate majority and minority classes
df_majority = df[df.churn_Y==0]
df_minority = df[df.churn_Y==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=600,    # to match majority class
                                 random_state=123) # reproducible results

# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=1400,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])
 
# Display new class counts
df_downsampled.churn_Y.value_counts()

0    1400
1     397
Name: churn_Y, dtype: int64

In [44]:
df = df_downsampled

In [45]:
from sklearn.linear_model import LogisticRegression

cols = list(df.columns)
cols.remove('churn_Y')
y = df['churn_Y']
X = df[cols]

logReg = LogisticRegression()
logReg.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [46]:
import pickle 

pickle.dump(logReg, open('/mnt/output/baseline.pkl', 'wb'))